In [1]:
import numpy as np

class FederatedLearningDemo:
    def __init__(self, num_clients):
        self.num_clients = num_clients
        self.global_model = None
        self.client_data = []

    def generate_data(self):
        for _ in range(self.num_clients):
            X = np.random.rand(100, 1)
            y = 2 * X + np.random.randn(100, 1) * 0.1
            self.client_data.append((X, y))

    def initialize_global_model(self):
        self.global_model = np.random.randn(1, 1)

    def train_local_model(self, client_id):
        X, y = self.client_data[client_id]
        local_model = np.copy(self.global_model)
        
        for _ in range(100):  # Local training iterations
            gradient = np.mean(X * (X.dot(local_model) - y))
            local_model -= 0.01 * gradient
        
        return local_model

    def aggregate_models(self, local_models):
        return np.mean(local_models, axis=0)

    def federated_learning(self, num_rounds):
        self.generate_data()
        self.initialize_global_model()

        for round in range(num_rounds):
            local_models = []
            for client_id in range(self.num_clients):
                local_model = self.train_local_model(client_id)
                local_models.append(local_model)
            
            self.global_model = self.aggregate_models(local_models)
            
            mse = self.evaluate_global_model()
            print(f"Round {round + 1}, Global Model MSE: {mse:.4f}")

    def evaluate_global_model(self):
        mse = 0
        for X, y in self.client_data:
            predictions = X.dot(self.global_model)
            mse += np.mean((predictions - y) ** 2)
        return mse / self.num_clients

# Run the demo
fl_demo = FederatedLearningDemo(num_clients=5)
fl_demo.federated_learning(num_rounds=10)

Round 1, Global Model MSE: 0.5343
Round 2, Global Model MSE: 0.2882
Round 3, Global Model MSE: 0.1572
Round 4, Global Model MSE: 0.0875
Round 5, Global Model MSE: 0.0504
Round 6, Global Model MSE: 0.0307
Round 7, Global Model MSE: 0.0201
Round 8, Global Model MSE: 0.0145
Round 9, Global Model MSE: 0.0116
Round 10, Global Model MSE: 0.0100


In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class FlowerFederatedLearning:
    def __init__(self, num_clients):
        self.num_clients = num_clients
        self.global_model = None
        self.client_data = []
        self.scaler = StandardScaler()

    def generate_data(self):
        # Generate synthetic data for three flower types
        n_samples_per_class = 333  # Ensure it's divisible by 3
        n_features = 2  # petal length and width

        # Rose: longer petals
        rose = np.random.randn(n_samples_per_class, n_features) * 0.5 + [5, 2]
        
        # Tulip: medium petals
        tulip = np.random.randn(n_samples_per_class, n_features) * 0.5 + [3, 1.5]
        
        # Daisy: shorter petals
        daisy = np.random.randn(n_samples_per_class, n_features) * 0.5 + [2, 1]

        X = np.vstack([rose, tulip, daisy])
        y = np.array([0] * n_samples_per_class + [1] * n_samples_per_class + [2] * n_samples_per_class)

        # Shuffle the data
        n_samples = len(X)
        shuffle_idx = np.random.permutation(n_samples)
        X, y = X[shuffle_idx], y[shuffle_idx]

        X = self.scaler.fit_transform(X)
        
        # Split data among clients
        for _ in range(self.num_clients):
            X_client, X_temp, y_client, y_temp = train_test_split(X, y, test_size=0.8, stratify=y)
            X, y = X_temp, y_temp
            self.client_data.append((X_client, y_client))

    def initialize_global_model(self):
        self.global_model = np.random.randn(2, 3)  # 2 features, 3 classes

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def train_local_model(self, client_id):
        X, y = self.client_data[client_id]
        local_model = np.copy(self.global_model)
        learning_rate = 0.01
        
        for _ in range(100):  # Local training iterations
            z = X.dot(local_model)
            y_pred = self.softmax(z)
            
            # One-hot encode true labels
            y_true = np.eye(3)[y]
            
            # Gradient
            gradient = X.T.dot(y_pred - y_true) / len(X)
            
            # Update local model
            local_model -= learning_rate * gradient
        
        return local_model

    def aggregate_models(self, local_models):
        return np.mean(local_models, axis=0)

    def federated_learning(self, num_rounds):
        self.generate_data()
        self.initialize_global_model()

        for round in range(num_rounds):
            local_models = []
            for client_id in range(self.num_clients):
                local_model = self.train_local_model(client_id)
                local_models.append(local_model)
            
            self.global_model = self.aggregate_models(local_models)
            
            accuracy = self.evaluate_global_model()
            print(f"Round {round + 1}, Global Model Accuracy: {accuracy:.4f}")

    def evaluate_global_model(self):
        correct_predictions = 0
        total_samples = 0
        
        for X, y in self.client_data:
            z = X.dot(self.global_model)
            y_pred = np.argmax(self.softmax(z), axis=1)
            correct_predictions += np.sum(y_pred == y)
            total_samples += len(y)
        
        return correct_predictions / total_samples

# Run the demo
np.random.seed(42)
fl_demo = FlowerFederatedLearning(num_clients=5)
fl_demo.federated_learning(num_rounds=20)

Round 1, Global Model Accuracy: 0.1878
Round 2, Global Model Accuracy: 0.3651
Round 3, Global Model Accuracy: 0.6289
Round 4, Global Model Accuracy: 0.7243
Round 5, Global Model Accuracy: 0.7437
Round 6, Global Model Accuracy: 0.7541
Round 7, Global Model Accuracy: 0.7645
Round 8, Global Model Accuracy: 0.7660
Round 9, Global Model Accuracy: 0.7601
Round 10, Global Model Accuracy: 0.7511
Round 11, Global Model Accuracy: 0.7556
Round 12, Global Model Accuracy: 0.7511
Round 13, Global Model Accuracy: 0.7481
Round 14, Global Model Accuracy: 0.7481
Round 15, Global Model Accuracy: 0.7437
Round 16, Global Model Accuracy: 0.7407
Round 17, Global Model Accuracy: 0.7392
Round 18, Global Model Accuracy: 0.7422
Round 19, Global Model Accuracy: 0.7422
Round 20, Global Model Accuracy: 0.7407
